In [8]:
import os
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnableSequence, RunnablePassthrough
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
apikey = os.environ['GROQ_API_KEY']

In [2]:
# Create a chat prompt template for goal-setting
llm = ChatGroq(
    model_name="llama-3.1-8b-Instant",  # or "mixtral-8x7b-32768" or another available model
    temperature=0.7,
    max_tokens=2048
)


In [ ]:
# Initialize embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
# Set up the persistent Chroma vector store
PERSIST_DIRECTORY = "data"

In [ ]:
vectorstore = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embeddings)

In [ ]:
client = chromadb.PersistentClient(path = "data")

In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant that helps users break down their goals into actionable steps."),
    ("human", "I want to achieve the following goal: {current_goal}. Can you help me break this down into smaller, actionable steps?"),
])


In [4]:
# Set up conversation memory
message_history = ChatMessageHistory()
memory = ConversationBufferMemory(
    chat_memory=message_history,
    return_messages=True,
    output_key="output"
)

In [5]:
# Create a chain for goal-setting
def load_memory(input_dict):
    memory_variables = memory.load_memory_variables({})
    return {"chat_history": memory_variables["history"]}

In [6]:
chain = (
    RunnablePassthrough.assign(chat_history=load_memory)
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
# Example usage
user_goal = "  "
response = chain.invoke({"current_goal": user_goal})

print(response)

In [ ]:
# Save the conversation to memory
memory.save_context({"input": user_goal}, {"output": response})

# Example of retrieving memory
print("\nConversation History:")
for message in message_history.messages:
    print(f"{message.type}: {message.content}")

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [ ]:
print(customer_messages[0])

In [ ]:
customer_response = chat(customer_messages)

In [ ]:
print(customer_response.content)

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

In [ ]:
%pip install --upgrade --quiet langchain-community arxiv